In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [7]:
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

#Load datsets
train_loader = torch.utils.data.DataLoader(datasets.USPS(root=".", train=True, download=True, transform=transform), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.USPS(root=".", train=False, download=True, transform=transform), batch_size=64, shuffle=False)

100%|██████████| 6579383/6579383 [00:01<00:00, 4092109.83it/s]


100%|██████████| 1831726/1831726 [00:01<00:00, 1476951.31it/s]


In [8]:
# MLP Architecture
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(16*16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 16*16)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [9]:
# CNN Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Add changes in Assignment 2 - Changes in kernel
        self.conv1 = nn.Conv2d(1, 32, kernel_size=4, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [11]:
#Initialization
mlp_model1 = MLP()
cnn_model2 = CNN()

In [12]:
#Optimizer
criterion = nn.NLLLoss()
# Add changes in Assignment 2 - Chaning in the learning rate
mlp_optimizer = optim.Adam(mlp_model1.parameters(), lr=0.0012)
# Add changes in Assignment 2 - Chaning in the learning rate
cnn_optimizer = optim.Adam(cnn_model2.parameters(), lr=0.0012)

In [13]:
#MLP Model Training 
mlp_model1.train()
for epoch in range(10):
    r_loss = 0.0
    for i in train_loader:
        inputs, labels = i
        mlp_optimizer.zero_grad()
        outputs = mlp_model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        mlp_optimizer.step()
        r_loss += loss.item()
    print(f'Multilayer Perceptron: Epoch [{epoch+1}/10], Loss: {r_loss/len(train_loader):.4f}')

Multilayer Perceptron: Epoch [1/10], Loss: 0.5431
Multilayer Perceptron: Epoch [2/10], Loss: 0.1747
Multilayer Perceptron: Epoch [3/10], Loss: 0.1233
Multilayer Perceptron: Epoch [4/10], Loss: 0.0896
Multilayer Perceptron: Epoch [5/10], Loss: 0.0741
Multilayer Perceptron: Epoch [6/10], Loss: 0.0590
Multilayer Perceptron: Epoch [7/10], Loss: 0.0422
Multilayer Perceptron: Epoch [8/10], Loss: 0.0369
Multilayer Perceptron: Epoch [9/10], Loss: 0.0268
Multilayer Perceptron: Epoch [10/10], Loss: 0.0200


In [14]:
#Conbvolution Neural Network Model Training
cnn_model2.train()
for epoch in range(10):
    r1_loss = 0.0
    for j in train_loader:
        inputs, labels = j
        cnn_optimizer.zero_grad()
        outputs = cnn_model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        cnn_optimizer.step()
        r1_loss += loss.item()
    print(f'Convolution Neural Network: epoch [{epoch+1}/10], Loss: {r1_loss/len(train_loader):.4f}')

Convolution Neural Network: epoch [1/10], Loss: 0.4611
Convolution Neural Network: epoch [2/10], Loss: 0.1057
Convolution Neural Network: epoch [3/10], Loss: 0.0744
Convolution Neural Network: epoch [4/10], Loss: 0.0439
Convolution Neural Network: epoch [5/10], Loss: 0.0403
Convolution Neural Network: epoch [6/10], Loss: 0.0241
Convolution Neural Network: epoch [7/10], Loss: 0.0267
Convolution Neural Network: epoch [8/10], Loss: 0.0179
Convolution Neural Network: epoch [9/10], Loss: 0.0127
Convolution Neural Network: epoch [10/10], Loss: 0.0187


In [15]:
#Evaluation for MLP and CNN
mlp_model1.eval()
c_mlp = 0
t_mlp = 0
with torch.no_grad():
    for k in test_loader:
        inputs, labels = k
        outputs = mlp_model1(inputs)
        _, predicted = torch.max(outputs.data, 1)
        t_mlp += labels.size(0)
        c_mlp += (predicted == labels).sum().item()

print(f'Multilayer perceptron Accuracy on test data set: {100 * c_mlp / t_mlp:.2f}%')

#CNN model
cnn_model2.eval()
c1_cnn = 0
t1_cnn = 0
with torch.no_grad():
    for l in test_loader:
        inputs, labels = l
        outputs = cnn_model2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        t1_cnn += labels.size(0)
        c1_cnn += (predicted == labels).sum().item()

print(f'CNN Accuracy on test data set: {100 * c1_cnn / t1_cnn:.2f}%')

Multilayer perceptron Accuracy on test data set: 93.82%
CNN Accuracy on test data set: 96.11%


In [17]:
# MOdel Evaluation
def evaluate_model(model, data_loader):
    predictions = []
    true_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    cm = confusion_matrix(true_labels, predictions)

    return accuracy, precision, recall, cm

In [18]:
mlp_accuracy, mlp_precision, mlp_recall, mlp_cm = evaluate_model(mlp_model1, test_loader)

cnn_accuracy, cnn_precision, cnn_recall, cnn_cm = evaluate_model(cnn_model2, test_loader)

#Evaluation Metrices
print("Multilayer perceptron Model:")
print(f"Accuracy: {mlp_accuracy:.4f}")
print(f"Precision: {mlp_precision:.4f}")
print(f"Recall: {mlp_recall:.4f}")
print("Confusion Matrix:")
print(mlp_cm)
print()
print("Convolution Neural Network Model:")
print(f"Accuracy: {cnn_accuracy:.4f}")
print(f"Precision: {cnn_precision:.4f}")
print(f"Recall: {cnn_recall:.4f}")
print("Confusion Matrix:")
print(cnn_cm)

Multilayer perceptron Model:
Accuracy: 0.9382
Precision: 0.9393
Recall: 0.9382
Confusion Matrix:
[[346   0   2   0   3   0   6   0   1   1]
 [  0 254   1   3   0   0   5   0   1   0]
 [  1   0 181   2   1   3   1   1   7   1]
 [  0   0   3 142   0  13   0   1   6   1]
 [  1   2   3   1 183   1   3   1   1   4]
 [  2   0   0   3   1 149   1   0   2   2]
 [  0   0   3   0   2   1 164   0   0   0]
 [  0   0   1   0   6   0   0 135   3   2]
 [  1   0   2   4   0   1   0   1 155   2]
 [  0   0   0   1   1   0   0   0   1 174]]

Convolution Neural Network Model:
Accuracy: 0.9611
Precision: 0.9623
Recall: 0.9611
Confusion Matrix:
[[351   0   1   2   2   0   2   0   0   1]
 [  0 258   0   0   5   0   0   1   0   0]
 [  1   0 179   5   3   1   0   1   8   0]
 [  1   0   0 157   0   7   0   0   1   0]
 [  0   2   0   0 193   1   0   1   0   3]
 [  1   0   0   4   0 152   0   0   1   2]
 [  0   1   0   0   4   0 165   0   0   0]
 [  0   0   0   1   7   0   0 139   0   0]
 [  0   0   0   0   1   3

## Tenspor Board

In [19]:
from torch.utils.tensorboard import SummaryWriter

In [20]:
mlp_writer = SummaryWriter('logs/mlp')
cnn_writer = SummaryWriter('logs/cnn')

In [21]:
def evaluate_tensorboard_model(model, data_loader, writer, epoch):
    predictions = []
    true_labels = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    cm = confusion_matrix(true_labels, predictions)
    # Log metrics
    writer.add_scalar('Accuracy', accuracy, epoch)
    writer.add_scalar('Precision', precision, epoch)
    writer.add_scalar('Recall', recall, epoch)
    writer.add_pr_curve('Precision-Recall Curve', torch.tensor(true_labels), torch.tensor(predictions), epoch)

    return accuracy, precision, recall, cm

In [22]:
mlp_accuracy, mlp_precision, mlp_recall, mlp_cm = evaluate_tensorboard_model(mlp_model1, test_loader, mlp_writer, 0)

cnn_accuracy, cnn_precision, cnn_recall, cnn_cm = evaluate_tensorboard_model(cnn_model2, test_loader, cnn_writer, 0)
mlp_writer.close()
cnn_writer.close()

# TensorBoard Accuracy
print("MLP:")
print(f"Accuracy: {mlp_accuracy:.4f}")
print(f"Precision: {mlp_precision:.4f}")
print(f"Recall: {mlp_recall:.4f}")
print("Confusion Matrix:")
print(mlp_cm)
print()
print("CNN :")
print(f"Accuracy: {cnn_accuracy:.4f}")
print(f"Precision: {cnn_precision:.4f}")
print(f"Recall: {cnn_recall:.4f}")
print("Confusion Matrix:")
print(cnn_cm)

MLP Model:
Accuracy: 0.9382
Precision: 0.9393
Recall: 0.9382
Confusion Matrix:
[[346   0   2   0   3   0   6   0   1   1]
 [  0 254   1   3   0   0   5   0   1   0]
 [  1   0 181   2   1   3   1   1   7   1]
 [  0   0   3 142   0  13   0   1   6   1]
 [  1   2   3   1 183   1   3   1   1   4]
 [  2   0   0   3   1 149   1   0   2   2]
 [  0   0   3   0   2   1 164   0   0   0]
 [  0   0   1   0   6   0   0 135   3   2]
 [  1   0   2   4   0   1   0   1 155   2]
 [  0   0   0   1   1   0   0   0   1 174]]

CNN Model:
Accuracy: 0.9611
Precision: 0.9623
Recall: 0.9611
Confusion Matrix:
[[351   0   1   2   2   0   2   0   0   1]
 [  0 258   0   0   5   0   0   1   0   0]
 [  1   0 179   5   3   1   0   1   8   0]
 [  1   0   0 157   0   7   0   0   1   0]
 [  0   2   0   0 193   1   0   1   0   3]
 [  1   0   0   4   0 152   0   0   1   2]
 [  0   1   0   0   4   0 165   0   0   0]
 [  0   0   0   1   7   0   0 139   0   0]
 [  0   0   0   0   1   3   0   0 160   2]
 [  0   0   0   0   0  